In [1]:
import os
import pandas as pd
import numpy as np
import random

import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#import matplotlib.pyplot as plt

import keras
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

from tensorflow.python.client import device_lib
import ast
from ast import literal_eval
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


c:\Users\l\anaconda3\envs\koo\lib\site-packages\tensorflow\python\framework\dtypes.py:513: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [13]:
def get_clf_eval(y_test, pred):
    score = []
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    specificity = tn/(tn + fp)
    roc_auc = roc_auc_score(y_test, pred)

    score = [accuracy, precision, recall, f1, specificity, roc_auc]
    return score
def get_mean(accuracy, precision, recall, f1,specificity,roc_auc):
  print("accuracy: ")
  print(np.mean(accuracy))
  print(np.std(accuracy))
  print("precision: ")
  print(np.mean(precision))
  print(np.std(precision))
  print("recall: ")
  print(np.mean(recall))
  print(np.std(recall))
  print("f1: ")
  print(np.mean(f1))
  print(np.std(f1))
  print("specificity: ")
  print(np.mean(specificity))
  print(np.std(specificity))
  print("roc_auc: ")
  print(np.mean(roc_auc))
  print(np.std(roc_auc))
  return print("end")
   
 

In [3]:
df = pd.read_csv('final_use.csv', encoding = 'utf-8-sig', index_col = 0)

In [4]:
df['preprocessing'] = df['preprocessing'].apply(lambda x: ast.literal_eval(x))
#text_num = 100
#df['preprocessing'] = df['preprocessing'].apply(lambda x: to_string(x))

KeyboardInterrupt: 

In [ ]:
vocab_size = 48972
text_num = 92

In [ ]:
def preprocess(df, vocab_size, text_num):
    X = df[['rating', 'preprocessing', 'depth',
       'text_structure', 'review_extremity', 'reputation', 'experience',
       'location', 'Psychological Distancing', 'style', 'photo',
        'motion', 'Affect', 'Cognition', 'Perception', 'Social']] 
    y = df[['fake']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 215, stratify=y)
    
    tokenizer = Tokenizer(vocab_size)
    tokenizer.fit_on_texts(X_train['preprocessing'].astype(str))
    X_train['text_encode'] = tokenizer.texts_to_sequences(X_train['preprocessing'].astype(str))
#     length = [len(sample) for sample in X_train['text_encode']]
#     length.sort()
    #text_num = length[int(len(length)*0.9)]
    train_text = pad_sequences(X_train['text_encode'].values.tolist(), maxlen=text_num)
    
    #linguistic feature
    
    train_affect = np.array(X_train['Affect'].values.tolist()).reshape((-1, 1))
    train_motion = np.array(X_train['motion'].values.tolist()).reshape((-1, 1))
    train_cognition = np.array(X_train['Cognition'].values.tolist()).reshape((-1, 1))
    train_perception = np.array(X_train['Perception'].values.tolist()).reshape((-1, 1))
    train_social = np.array(X_train['Social'].values.tolist()).reshape((-1, 1))
    train_p_dis = np.array(X_train['Psychological Distancing'].values.tolist()).reshape((-1, 1))
    
    #review feature
    
    train_rating = np.array(X_train['rating'].values.tolist()).reshape((-1, 1))
    train_depth = np.array(X_train['depth'].values.tolist()).reshape((-1, 1))
    train_structure = np.array(X_train['text_structure'].values.tolist()).reshape((-1, 1))
    train_extremity = np.array(X_train['review_extremity'].values.tolist()).reshape((-1, 1))
    train_photo = np.array(X_train['photo'].values.tolist()).reshape((-1, 1))
    train_style = np.array(X_train['style'].values.tolist()).reshape((-1, 1))
    
    #reviewer feature
    
    train_reputation = np.array(X_train['reputation'].values.tolist()).reshape((-1, 1))
    train_experience = np.array(X_train['experience'].values.tolist()).reshape((-1, 1))
    train_location = np.array(X_train['location'].values.tolist()).reshape((-1, 1))
    
    train_fake = np.array(y_train.values.tolist()).reshape((-1, 1))
    
    
    #linguistic feature
    
    X_test['text_encode'] = tokenizer.texts_to_sequences(X_test['preprocessing'].astype(str))
    test_text = pad_sequences(X_test['text_encode'].values.tolist(), maxlen=text_num)
    test_affect = np.array(X_test['Affect'].values.tolist()).reshape((-1, 1))
    test_motion = np.array(X_test['motion'].values.tolist()).reshape((-1, 1))
    test_cognition = np.array(X_test['Cognition'].values.tolist()).reshape((-1, 1))
    test_perception = np.array(X_test['Perception'].values.tolist()).reshape((-1, 1))
    test_social = np.array(X_test['Social'].values.tolist()).reshape((-1, 1))
    test_p_dis = np.array(X_test['Psychological Distancing'].values.tolist()).reshape((-1, 1))
    
    #review feature
    
    test_rating = np.array(X_test['rating'].values.tolist()).reshape((-1, 1))
    test_depth = np.array(X_test['depth'].values.tolist()).reshape((-1, 1))
    test_structure = np.array(X_test['text_structure'].values.tolist()).reshape((-1, 1))
    test_extremity = np.array(X_test['review_extremity'].values.tolist()).reshape((-1, 1))
    test_photo = np.array(X_test['photo'].values.tolist()).reshape((-1, 1))
    test_style = np.array(X_test['style'].values.tolist()).reshape((-1, 1))
    
    #reviewer feature
    
    test_reputation = np.array(X_test['reputation'].values.tolist()).reshape((-1, 1))
    test_experience = np.array(X_test['experience'].values.tolist()).reshape((-1, 1))
    test_location = np.array(X_test['location'].values.tolist()).reshape((-1, 1))
    
    test_fake = np.array(y_test.values.tolist()).reshape((-1, 1))
    
    return train_text, train_affect, train_motion, train_cognition, train_perception, train_social, train_p_dis, train_rating, \
train_depth, train_structure, train_extremity, train_photo, train_style, train_reputation, train_experience, train_location, train_fake, \
test_text, test_affect, test_motion, test_cognition, test_perception, test_social, test_p_dis, test_rating, test_depth, test_structure, \
test_extremity, test_photo, test_style, test_reputation, test_experience, test_location, test_fake
    

In [7]:
train_text, train_affect, train_motion, train_cognition, train_perception, train_social, train_p_dis, train_rating, \
train_depth, train_structure, train_extremity, train_photo, train_style, train_reputation, train_experience, train_location, train_fake, \
test_text, test_affect, test_motion, test_cognition, test_perception, test_social, test_p_dis, test_rating, test_depth, test_structure, \
test_extremity, test_photo, test_style, test_reputation, test_experience, test_location, test_fake= preprocess(df, vocab_size, text_num)

In [ ]:
train_text = X_train['preprocessing']
test_text = X_test['preprocessing']

#CBOW
sentences = [sentence.split() for sentence in train_text]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)


# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
vocab_size = len(tokenizer.word_index) + 1
train_text = tokenizer.texts_to_sequences(train_text)
test_text = tokenizer.texts_to_sequences(test_text)

vocab_size = len(tokenizer.word_index) + 1

# Pad the sequences to a fixed length
max_length = 100
train_text = pad_sequences(train_text, maxlen=max_length)
test_text = pad_sequences(test_text, maxlen=max_length)

# Train the Word2Vec model


# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
embedding_layer = Embedding(vocab_size, 100, weights = [embedding_matrix])

In [ ]:
train_text = X_train['preprocessing']
test_text = X_test['preprocessing']

#glove
sentences = [sentence.split() for sentence in train_text]
#w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
corpus = Corpus()
corpus.fit(sentences, window=5)
glove = Glove(no_components=100, learning_rate=0.0001)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=50, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

word_dict = {}
for word in  glove.dictionary.keys():
    word_dict[word] = glove.word_vectors[glove.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
vocab_size = len(tokenizer.word_index) + 1
train_text = tokenizer.texts_to_sequences(train_text)
test_text = tokenizer.texts_to_sequences(test_text)

vocab_size = len(tokenizer.word_index) + 1

# Pad the sequences to a fixed length
max_length = 100
train_text = pad_sequences(train_text, maxlen=max_length)
test_text = pad_sequences(test_text, maxlen=max_length)

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word_dict.keys():
    	embedding_matrix[i] = word_dict.get(word, np.zeros(100))
     
embedding_layer = Embedding(vocab_size, 100, weights = [embedding_matrix])

In [8]:
#CNN 있는 거1
def Model_Build(text_num, vocab_size):
    
    tf.random.set_seed(215)
    
    #CNN
    text_input = Input(shape=(text_num, ))
    text_embedding = Embedding(vocab_size, 100, input_length = text_input.shape[1], name="text-embedding")(text_input)

#    text_embedding = embedding_layer(text_input)
    text_dropout = Dropout(0.2)(text_embedding)
    text_conv = Conv1D(256, 3, padding='valid', activation='relu')(text_dropout)
    text_pool = GlobalMaxPooling1D()(text_conv)
    text_pool = Dense(units = 128, activation='relu')(text_pool)
    
    #linguistic
    affect_input = Input(shape = (1,))
    dense_affect = Dense(units=64, activation='relu', name='dense_affect')(affect_input)
    motion_input = Input(shape = (1,))
    dense_motion = Dense(units=64, activation='relu', name='motion')(motion_input)
    cognitive_input = Input(shape = (1,))
    dense_cognitive = Dense(units=64, activation='relu', name='dense_cognitive')(cognitive_input)
    perception_input = Input(shape = (1,))
    dense_perception = Dense(units=64, activation='relu', name='dense_perception')(perception_input)
    social_input = Input(shape = (1,))
    dense_social = Dense(units=64, activation='relu', name='dense_social')(social_input)
    p_dis_input = Input(shape = (1,))
    dense_p_dis = Dense(units=64, activation='relu', name='dense_p_dis')(p_dis_input)

    linguistic_feature = Concatenate(name = 'Linguistic_Feature_Layer')([dense_affect, dense_motion, dense_cognitive, dense_perception, dense_social, dense_p_dis])
    linguistic_feature1 = Dense(units=128, activation='relu', name='dense_linguistic')(linguistic_feature)

    #review
    rating_input = Input(shape = (1,))
    dense_rating = Dense(units=64, activation='relu', name='dense_rating')(rating_input)
    depth_input = Input(shape = (1,))
    dense_depth = Dense(units=64, activation='relu', name='dense_depth')(depth_input)
    structure_input = Input(shape = (1,))
    dense_structure = Dense(units=64, activation='relu', name='dense_structure')(structure_input)
    extremity_input = Input(shape = (1,))
    dense_extremity = Dense(units=64, activation='relu', name='dense_extremity')(extremity_input)
    photo_input = Input(shape = (1,))
    dense_photo = Dense(units=64, activation='relu', name='dense_photo')(photo_input)
    style_input = Input(shape = (1,))
    dense_style = Dense(units=64, activation='relu', name='dense_style')(style_input)

    review_feature = Concatenate(name = 'review_Feature_Layer')([dense_rating, dense_depth, dense_structure, dense_extremity, dense_photo, dense_style])
    review_feature1 = Dense(units=128, activation='relu', name='dense_review')(review_feature)

    #reviewer
    
    reputation_input = Input(shape = (1,))
    dense_reputation = Dense(units=64, activation='relu', name='dense_reputation')(reputation_input)
    experience_input = Input(shape = (1,))
    dense_experience = Dense(units=64, activation='relu', name='dense_experience')(experience_input)
    location_input = Input(shape = (1,))
    dense_location = Dense(units=64, activation='relu', name='dense_location')(location_input)
    
    reviewer_feature = Concatenate(name = 'reviewer_Feature_Layer')([dense_reputation, dense_location, dense_experience])
    reviewer_feature1 = Dense(units=128, activation='relu', name='dense_reviewer')(reviewer_feature)

    all_feature = Concatenate(name = 'All_Feature_Layer')([text_pool, reviewer_feature1, review_feature1, linguistic_feature1])
    #all_feature1 = Dense(units=256, activation='relu', name='dense_all')(all_feature)
    #all_feature2 = Dense(units=128, activation='relu', name='dense_all1')(all_feature)
    all_feature3 = Dense(units=64, activation='relu', name='dense_all2')(all_feature)
    all_feature4 = Dense(units=32, activation='relu', name='dense_all3')(all_feature3)
    all_feature5 = Dense(units=16, activation='relu', name='dense_all4')(all_feature4)

    
    outputs = Dense(1, activation='sigmoid', name = 'outputs')(all_feature5)  # (1,1) / h(8,1)초기화
    model = Model(inputs=[text_input, affect_input, motion_input, cognitive_input, perception_input,social_input, p_dis_input, rating_input, depth_input, structure_input, extremity_input, photo_input, style_input, reputation_input, experience_input, location_input], outputs=outputs)
    return model

In [9]:
score4 = []
acc = []
precision = []
recall=[]
f1=[]
spe = []
roc = []
for i in range(5):
    model = Model_Build(text_num, vocab_size)

    adam = Adam(learning_rate=0.0001)

    model.compile(optimizer=adam, loss='binary_crossentropy')

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)


    history = model.fit([train_text, train_affect, train_motion, train_cognition, train_perception, train_social, train_p_dis, train_rating, train_depth, train_structure, train_extremity, train_photo, train_style, train_reputation, train_experience, train_location], 
                train_fake, 
                batch_size = 2048, 
                epochs = 100,
                callbacks=[es],
                validation_split = 0.2)

    prediction4 = model.predict([test_text, test_affect, test_motion, test_cognition, test_perception, test_social, test_p_dis, test_rating, test_depth, test_structure, test_extremity, test_photo, test_style, test_reputation, test_experience, test_location])

    pred_class4 = np.where(prediction4 >= 0.5, 1 , 0)
    score4 = get_clf_eval(test_fake, pred_class4)
    acc.append(score4[0])
    precision.append(score4[1])
    recall.append(score4[2])
    f1.append(score4[3])
    spe.append(score4[4])
    roc.append(score4[5])
    print(score4)
print(get_mean(acc, precision, recall, f1, spe, roc))

Epoch 1/100
142/142 [==============================] - 65s 449ms/step - loss: 0.3541 - val_loss: 0.2342
Epoch 2/100
142/142 [==============================] - 66s 464ms/step - loss: 0.2252 - val_loss: 0.2213
Epoch 3/100
 18/142 [==>...........................] - ETA: 55s - loss: 0.2158

KeyboardInterrupt: 

In [ ]:
#CNN만
def Model_Build(text_num, vocab_size):
    
    tf.random.set_seed(215)
    
    #CNN
    text_input = Input(shape=(text_num, ))
    text_embedding = Embedding(vocab_size, 100, input_length = text_input.shape[1], name="text-embedding")(text_input)
    text_dropout = Dropout(0.2)(text_embedding)
    text_conv = Conv1D(256, 3, padding='valid', activation='relu')(text_dropout)
    text_pool = GlobalMaxPooling1D()(text_conv)
    text_pool = Dense(units = 128, activation='relu')(text_pool)
    
    #review
    rating_input = Input(shape = (1,))
    dense_rating = Dense(units=64, activation='relu', name='dense_rating')(rating_input)
    depth_input = Input(shape = (1,))
    dense_depth = Dense(units=64, activation='relu', name='dense_depth')(depth_input)
    structure_input = Input(shape = (1,))
    dense_structure = Dense(units=64, activation='relu', name='dense_structure')(structure_input)
    extremity_input = Input(shape = (1,))
    dense_extremity = Dense(units=64, activation='relu', name='dense_extremity')(extremity_input)
    photo_input = Input(shape = (1,))
    dense_photo = Dense(units=64, activation='relu', name='dense_photo')(photo_input)
    style_input = Input(shape = (1,))
    dense_style = Dense(units=64, activation='relu', name='dense_style')(style_input)

    review_feature = Concatenate(name = 'review_Feature_Layer')([dense_rating, dense_depth, dense_structure, dense_extremity, dense_photo, dense_style])
    review_feature1 = Dense(units=128, activation='relu', name='dense_review')(review_feature)

    #reviewer
    
    reputation_input = Input(shape = (1,))
    dense_reputation = Dense(units=64, activation='relu', name='dense_reputation')(reputation_input)
    experience_input = Input(shape = (1,))
    dense_experience = Dense(units=64, activation='relu', name='dense_experience')(experience_input)
    location_input = Input(shape = (1,))
    dense_location = Dense(units=64, activation='relu', name='dense_location')(location_input)
    
    reviewer_feature = Concatenate(name = 'reviewer_Feature_Layer')([dense_reputation, dense_location, dense_experience])
    reviewer_feature1 = Dense(units=128, activation='relu', name='dense_reviewer')(reviewer_feature)

    all_feature = Concatenate(name = 'All_Feature_Layer')([text_pool, reviewer_feature1, review_feature1])
    #all_feature1 = Dense(units=256, activation='relu', name='dense_all')(all_feature)
    #all_feature2 = Dense(units=128, activation='relu', name='dense_all1')(all_feature)
    all_feature3 = Dense(units=64, activation='relu', name='dense_all2')(all_feature)
    all_feature4 = Dense(units=32, activation='relu', name='dense_all3')(all_feature3)
    all_feature5 = Dense(units=16, activation='relu', name='dense_all4')(all_feature4)

    
    outputs = Dense(1, activation='sigmoid', name = 'outputs')(all_feature5)  # (1,1) / h(8,1)초기화
    model = Model(inputs=[text_input, rating_input, depth_input, structure_input, extremity_input, photo_input, style_input, reputation_input, experience_input, location_input], outputs=outputs)
    return model

In [ ]:

score1 = []
for i in range(5):
    model = Model_Build(text_num, vocab_size)

    adam = Adam(learning_rate=0.0001)

    model.compile(optimizer=adam, loss='binary_crossentropy')

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)


    history = model.fit([train_text, train_rating, train_depth, train_structure, train_extremity, train_photo, train_style, train_reputation, train_experience, train_location], 
                train_fake, 
                batch_size = 2048, 
                epochs = 100,
                callbacks=[es],
                validation_split = 0.2)

    prediction = model.predict([test_text, test_rating, test_depth, test_structure, test_extremity, test_photo, test_style, test_reputation, test_experience, test_location])

    pred_class1 = np.where(prediction >= 0.5, 1 , 0)
    
    score1.append(get_clf_eval(test_fake, pred_class1))
    print(score1)
print(get_mean(score1))

Epoch 1/100
142/142 [==============================] - 18s 118ms/step - loss: 0.3538 - val_loss: 0.2375
Epoch 2/100
142/142 [==============================] - 17s 117ms/step - loss: 0.2278 - val_loss: 0.2218
Epoch 3/100
142/142 [==============================] - 17s 117ms/step - loss: 0.2154 - val_loss: 0.2118
Epoch 4/100
142/142 [==============================] - 17s 117ms/step - loss: 0.2048 - val_loss: 0.2033
Epoch 5/100
142/142 [==============================] - 17s 117ms/step - loss: 0.1953 - val_loss: 0.1985
Epoch 6/100
142/142 [==============================] - 17s 118ms/step - loss: 0.1884 - val_loss: 0.2002
Epoch 7/100
142/142 [==============================] - 17s 117ms/step - loss: 0.1835 - val_loss: 0.1958
Epoch 8/100
142/142 [==============================] - 17s 118ms/step - loss: 0.1790 - val_loss: 0.1950
Epoch 9/100
142/142 [==============================] - 17s 117ms/step - loss: 0.1758 - val_loss: 0.1976
Epoch 10/100
142/142 [==============================] - 17s 118m

In [12]:
#CNN 없는거
def Model_Build():
    
    tf.random.set_seed(215)
    
    
    #linguistic
    affect_input = Input(shape = (1,))
    dense_affect = Dense(units=64, activation='relu', name='dense_affect')(affect_input)
    motion_input = Input(shape = (1,))
    dense_motion = Dense(units=64, activation='relu', name='motion')(motion_input)
    cognitive_input = Input(shape = (1,))
    dense_cognitive = Dense(units=64, activation='relu', name='dense_cognitive')(cognitive_input)
    perception_input = Input(shape = (1,))
    dense_perception = Dense(units=64, activation='relu', name='dense_perception')(perception_input)
    social_input = Input(shape = (1,))
    dense_social = Dense(units=64, activation='relu', name='dense_social')(social_input)
    p_dis_input = Input(shape = (1,))
    dense_p_dis = Dense(units=64, activation='relu', name='dense_p_dis')(p_dis_input)

    linguistic_feature = Concatenate(name = 'Linguistic_Feature_Layer')([dense_affect, dense_motion, dense_cognitive, dense_perception, dense_social, dense_p_dis])
    linguistic_feature1 = Dense(units=128, activation='relu', name='dense_linguistic')(linguistic_feature)

    #review
    rating_input = Input(shape = (1,))
    dense_rating = Dense(units=64, activation='relu', name='dense_rating')(rating_input)
    depth_input = Input(shape = (1,))
    dense_depth = Dense(units=64, activation='relu', name='dense_depth')(depth_input)
    structure_input = Input(shape = (1,))
    dense_structure = Dense(units=64, activation='relu', name='dense_structure')(structure_input)
    extremity_input = Input(shape = (1,))
    dense_extremity = Dense(units=64, activation='relu', name='dense_extremity')(extremity_input)
    photo_input = Input(shape = (1,))
    dense_photo = Dense(units=64, activation='relu', name='dense_photo')(photo_input)
    style_input = Input(shape = (1,))
    dense_style = Dense(units=64, activation='relu', name='dense_style')(style_input)

    review_feature = Concatenate(name = 'review_Feature_Layer')([dense_rating, dense_depth, dense_structure, dense_extremity, dense_photo, dense_style])
    review_feature1 = Dense(units=128, activation='relu', name='dense_review')(review_feature)

    #reviewer
    
    reputation_input = Input(shape = (1,))
    dense_reputation = Dense(units=64, activation='relu', name='dense_reputation')(reputation_input)
    experience_input = Input(shape = (1,))
    dense_experience = Dense(units=64, activation='relu', name='dense_experience')(experience_input)
    location_input = Input(shape = (1,))
    dense_location = Dense(units=64, activation='relu', name='dense_location')(location_input)
    
    reviewer_feature = Concatenate(name = 'reviewer_Feature_Layer')([dense_reputation, dense_location, dense_experience])
    reviewer_feature1 = Dense(units=128, activation='relu', name='dense_reviewer')(reviewer_feature)

    all_feature = Concatenate(name = 'All_Feature_Layer')([reviewer_feature1, review_feature1, linguistic_feature1])
    #all_feature1 = Dense(units=256, activation='relu', name='dense_all')(all_feature)
    #all_feature2 = Dense(units=128, activation='relu', name='dense_all1')(all_feature)
    all_feature3 = Dense(units=64, activation='relu', name='dense_all2')(all_feature)
    all_feature4 = Dense(units=32, activation='relu', name='dense_all3')(all_feature3)
    all_feature5 = Dense(units=16, activation='relu', name='dense_all4')(all_feature4)

    
    outputs = Dense(1, activation='sigmoid', name = 'outputs')(all_feature5)  # (1,1) / h(8,1)초기화
    model = Model(inputs=[affect_input, motion_input, cognitive_input, perception_input,social_input, p_dis_input, rating_input, depth_input, structure_input, extremity_input, photo_input, style_input, reputation_input, experience_input, location_input], outputs=outputs)
    return model

In [13]:

score = []
for i in range(5):
    model = Model_Build()

    adam = Adam(learning_rate=0.0001)

    model.compile(optimizer=adam, loss='binary_crossentropy')

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)


    history = model.fit([train_affect, train_motion, train_cognition, train_perception, train_social, train_p_dis, train_rating, train_depth, train_structure, train_extremity, train_photo, train_style, train_reputation, train_experience, train_location], 
                train_fake, 
                batch_size = 2048, 
                epochs = 100,
                callbacks=[es],
                validation_split = 0.2)

    prediction = model.predict([test_affect, test_motion, test_cognition, test_perception, test_social, test_p_dis, test_rating, test_depth, test_structure, test_extremity, test_photo, test_style, test_reputation, test_experience, test_location])

    pred_class = np.where(prediction >= 0.5, 1 , 0)
    
    score.append(get_clf_eval(test_fake, pred_class))
    print(score)

print(get_mean(score))

Epoch 1/100
142/142 [==============================] - 3s 16ms/step - loss: 0.2407 - val_loss: 0.2248
Epoch 2/100
142/142 [==============================] - 2s 11ms/step - loss: 0.2187 - val_loss: 0.2149
Epoch 3/100
142/142 [==============================] - 2s 11ms/step - loss: 0.2120 - val_loss: 0.2114
Epoch 4/100
142/142 [==============================] - 2s 11ms/step - loss: 0.2073 - val_loss: 0.2057
Epoch 5/100
142/142 [==============================] - 2s 11ms/step - loss: 0.2037 - val_loss: 0.2029
Epoch 6/100
142/142 [==============================] - 2s 12ms/step - loss: 0.2012 - val_loss: 0.2025
Epoch 7/100
142/142 [==============================] - 2s 11ms/step - loss: 0.2003 - val_loss: 0.2043
Epoch 8/100
142/142 [==============================] - 2s 11ms/step - loss: 0.1999 - val_loss: 0.2033
Epoch 9/100
142/142 [==============================] - 2s 12ms/step - loss: 0.2000 - val_loss: 0.2006
Epoch 10/100
142/142 [==============================] - 2s 12ms/step - loss: 0.198